Импорт библиотек

In [7]:
import requests as r
import pandas as pd
import datetime as dt
import numpy as np

Задание 3

In [44]:
#Подавление ошибки цепного назначения
pd.options.mode.chained_assignment = None

#Раскрываем все строки и столбцы
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Запрашиваем данные через GET запрос и преобразуем в json
visits_json = r.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01').json()
registrations_json = r.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01').json()

#Преобразовываем json в DataFrame
visits = pd.DataFrame(visits_json)
registrations = pd.DataFrame(registrations_json)

#Добавляем столбец с датой
visits['date'] = pd.to_datetime(visits['datetime']).dt.date
registrations['date'] = pd.to_datetime(registrations['datetime']).dt.date

#Выбираем дату последнего визита каждого пользователя
visits["datetime"] = pd.to_datetime(visits["datetime"], format="%a, %d %b %Y %H:%M:%S %Z")
visits["last_datetime"] = visits.groupby('visit_id')["datetime"].transform('max')
last_visits = visits[visits['datetime'] == visits['last_datetime']]

#Маркируем ботов
# def is_bot(row):
#     return 1 if row['user_agent'].str.contains('bot') else 0

# last_visits['is_bot'] = last_visits['user_agent'].apply(is_bot, axis = 1)

last_visits['is_bot'] = np.where(last_visits['user_agent'].str.contains('bot'), 'yes', 'no')

#Группируем и объединяем данные
visits_and_registrations = pd.concat([
    last_visits[last_visits['is_bot'] == 'no'].groupby(['date', 'platform']).agg({'visit_id': 'count'}),
    registrations.groupby(['date', 'platform']).agg({'user_id': 'count'})
], axis=1)

#Переименовываем столбцы и заполняем нули
visits_and_registrations.rename(columns = {'visit_id':'visits', 'user_id':'registrations'}, inplace = True )
visits_and_registrations['registrations'] = visits_and_registrations['registrations'].fillna(0)

#Считаем конверсию
visits_and_registrations['conversion'] = round(visits_and_registrations['registrations']/visits_and_registrations['visits']*100, 0)

#Сохраняем в формате json
visits_and_registrations.to_json('conversion.json', orient='columns')

#print(visits_and_registrations)


                     visit_id
date       platform          
2023-03-01 android        264
           ios             78
           web            333
2023-03-02 android        106
           ios            103
           web            462
2023-03-03 android        251
           ios             57
           web           1221
2023-03-04 android        193
           ios            117
           web           1773
2023-03-05 android        287
           ios            110
           web           1924
2023-03-06 android        322
           ios            159
           web           2116
2023-03-07 android        172
           ios            186
           web           2580
2023-03-08 android        450
           ios            133
           web           1901
2023-03-09 android        106
           ios             83
           web           1937
2023-03-10 android        366
           ios             40
           web           2459
2023-03-11 android        222
          

Задание 4

In [48]:
#Читаем данные из csv файла
ads_info = pd.read_csv('ads.csv')

#Добавляем столбец с датой
ads_info['date_ymd'] = pd.to_datetime(ads_info['date']).dt.date

#Группируем по дате и рекламной компании, находим стоимость рекламных кампаний по дням
ads_grouped = ads_info.groupby(['date_ymd', 'utm_campaign']).agg({'cost':'sum'})

#Соединяем таблицу конверсий и рекламы
ads = visits_and_registrations.merge(ads_grouped, left_on = 'date', right_on = 'date_ymd', how = 'outer') 

print(ads.head(10))
print(visits_and_registrations.head(10))
print(ads_grouped.head(10))



   visits  registrations  conversion   cost
0   264.0          218.0        83.0  238.0
1    78.0           67.0        86.0  238.0
2   333.0           78.0        23.0  238.0
3   106.0           91.0        86.0  164.0
4   103.0           76.0        74.0  164.0
5   462.0           52.0        11.0  164.0
6   251.0          218.0        87.0  240.0
7    57.0           54.0        95.0  240.0
8  1221.0           66.0         5.0  240.0
9   193.0          119.0        62.0  167.0
                     visits  registrations  conversion
date       platform                                   
2023-03-01 android      264            218        83.0
           ios           78             67        86.0
           web          333             78        23.0
2023-03-02 android      106             91        86.0
           ios          103             76        74.0
           web          462             52        11.0
2023-03-03 android      251            218        87.0
           ios       